<a href="https://colab.research.google.com/github/ahmadsaad007/Deep_Learning_Projects/blob/master/Fundus_Image_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np
from keras.models import Sequential

Using TensorFlow backend.


In [0]:
import numpy as np
from tensorflow.keras import layers
from tensorflow import keras 
import tensorflow as tf
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.convolutional import *
from keras.utils import np_utils
from keras.optimizers import Adam
from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.layers.normalization import BatchNormalization
import itertools
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
%matplotlib inline
from keras import backend as K
from keras.applications.vgg16 import VGG16
from keras.layers import Input

In [3]:
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
train_path = "/content/drive/My Drive/Eye Disease Research/train"
test_path = "/content/drive/My Drive/Eye Disease Research/test"

imgdata = ImageDataGenerator( rescale=1.0/255.0, validation_split=0.10)
train_batches = imgdata.flow_from_directory(train_path,class_mode='categorical', subset='training', target_size=(768, 496) ,
                                            classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)
test_batches = imgdata.flow_from_directory(test_path, target_size=(768, 496) ,class_mode='categorical',
                                           classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)
valid_batches = imgdata.flow_from_directory(train_path, subset='validation', target_size=(768, 496) ,class_mode='categorical',
                                            classes = ['CNV','DRUSEN','NORMAL'], batch_size = 32)


Found 64937 images belonging to 3 classes.
Found 751 images belonging to 3 classes.
Found 7214 images belonging to 3 classes.


Lets try to see how many images do we have of each class and then use that to get the starting index of those images

In [6]:
mapper = {}
for label in train_batches.classes:
  if(label not in mapper):
    mapper[label] = 1
  else:
    val = mapper[label]
    mapper[label] = val+1
print(mapper)

{0: 33485, 1: 7764, 2: 23688}


# Let's train on a prebuilt model: VGG16:
We will need to reshape the input so that we can feed our images that are bigger than what the model was trained on.

In [7]:
vgg16_model = VGG16(input_tensor=Input(shape=(768, 496, 3)),input_shape=(768,496,3), include_top = False)
#vgg16_model.layers.pop()
vgg16_model.summary()

58892288/58889256 [==============================] - 1s 0us/step
Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 768, 496, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 768, 496, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 768, 496, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 384, 248, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 384, 248, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 384, 248, 128)     147584    
______________________________________________________________

## Create a sequential model and add all the layers in VGG16 to it. Then add the Dense layers for your custom output:

In [8]:
model= Sequential()
model.add(vgg16_model)
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 24, 15, 512)       14714688  
Total params: 14,714,688
Trainable params: 14,714,688
Non-trainable params: 0
_________________________________________________________________


In [9]:
#Make the vgg16's layers non-trainable
for layer in model.layers:
  layer.trainable = False
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(3, activation='softmax'))

model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
vgg16 (Model)                (None, 24, 15, 512)       14714688  
_________________________________________________________________
flatten_1 (Flatten)          (None, 184320)            0         
_________________________________________________________________
dense_1 (Dense)              (None, 256)               47186176  
_________________________________________________________________
dense_2 (Dense)              (None, 32)                8224      
_________________________________________________________________
dense_3 (Dense)              (None, 3)                 99        
Total params: 61,909,187
Trainable params: 47,194,499
Non-trainable params: 14,714,688
_________________________________________________________________


In [0]:
model.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [0]:
from sklearn.utils import class_weight
class_weights = class_weight.compute_class_weight('balanced', np.unique(train_batches.classes), 
                                                  train_batches.classes)

# **Training the modified VGG16 model on our data**

Epochs for training should be between 8 and 16 in my opinion with steps per epoch around 250 to 300.

Similarly Validation should have about 200 to 250 steps per epoch

**NOTE:** Numbers are kept lower than predicted because of lower computational power and resource available!

In [13]:
model.fit_generator(train_batches, steps_per_epoch=226, epochs=8, verbose=1, 
                    validation_data=valid_batches, validation_steps=64, 
                    shuffle=True, class_weight = class_weights)

Epoch 1/8
226/226 [==============================] - 1855s 8s/step - loss: 1.1723 - accuracy: 0.7640 - val_loss: 0.5525 - val_accuracy: 0.7832
Epoch 2/8
226/226 [==============================] - 1741s 8s/step - loss: 0.4665 - accuracy: 0.8518 - val_loss: 0.5109 - val_accuracy: 0.8501
Epoch 3/8
226/226 [==============================] - 1712s 8s/step - loss: 0.4168 - accuracy: 0.8609 - val_loss: 1.6516 - val_accuracy: 0.8086
Epoch 4/8
226/226 [==============================] - 1619s 7s/step - loss: 0.3291 - accuracy: 0.8815 - val_loss: 0.4787 - val_accuracy: 0.8754
Epoch 5/8
226/226 [==============================] - 1544s 7s/step - loss: 0.3150 - accuracy: 0.8841 - val_loss: 0.6184 - val_accuracy: 0.8560
Epoch 6/8
226/226 [==============================] - 1495s 7s/step - loss: 0.3104 - accuracy: 0.8897 - val_loss: 0.6018 - val_accuracy: 0.8970
Epoch 7/8
226/226 [==============================] - 1512s 7s/step - loss: 0.2533 - accuracy: 0.9118 - val_loss: 0.2918 - val_accuracy: 0.9062

## **Evaluate model on Test Data:**

In [16]:
acc = model.evaluate_generator(test_batches, steps=len(test_batches), verbose=1)
print(acc[1]*100)

24/24 [==============================] - 146s 6s/step
96.00532650947571


# **Save the Model:**

In [14]:
model.save("/content/drive/My Drive/Eye Disease Research/model_vgg16.h5")
print("Saved model to disk")

Saved model to disk


In [15]:
model_json = model.to_json()
with open("/content/drive/My Drive/Eye Disease Research/model_vgg16_jsn.json", "w") as json_file:
    json_file.write(model_json)
# serialize weights to HDF5
model.save_weights("model_vgg16_wt.h5")
print("Saved model to disk")

Saved model to disk


# **Let's train a Resnet50 model now and see how it performs on our data**

In [0]:
resnet_model = ResNet50(weights="imagenet",input_tensor=Input(shape=(768, 496, 3)),
                        input_shape=(768,496,3), include_top = False)
model_res= Sequential()
model_res.add(resnet_model)
for layer in model_res.layers:
  layer.trainable = False
model_res.add(Flatten())
model_res.add(Dense(128, activation='relu'))
model_res.add(Dense(16, activation='relu'))
model_res.add(Dense(3, activation='softmax'))

model_res.summary()

In [0]:
model_res.compile(Adam(learning_rate=0.001), loss='categorical_crossentropy', metrics=['accuracy'])
model_res.fit_generator(train_batches, steps_per_epoch=200, epochs=10,verbose=1, 
                    validation_data=valid_batches, validation_steps=32, shuffle=True, class_weight = class_weights)

## **Evaluate Model on Test Data:**

In [0]:
acc = model_res.evaluate_generator(test_batches, steps=len(test_batches), verbose=1)
print(acc[1]*100)

# **Save the modified Resnet50 Model:**

In [0]:
model_res.save("/content/drive/My Drive/Eye Disease Research/model_resnet50.h5")
print("Saved model to disk")

In [0]:
model_res_json = model_res.to_json()
with open("/content/drive/My Drive/Eye Disease Research/model_resnet50_jsn.json", "w") as json_file:
    json_file.write(model_res_json)
# serialize weights to HDF5
model_res.save_weights("model_resnet50_wt.h5")
print("Saved model to disk")